### Import library

SAAPpred script that predicts protein pathogenicty from SAAPdap data, using SciKit-Learn random forests. 
Goal is to predict SNP or PD with MCC > 0.7.
        
Best CV models are directly used for final testing

In [ ]:
""" Imports the required libraries and packages """

import pandas as pd                                                              # Data manipulation in dataframes
import numpy as np                                                               # Array manipulation
import pickle
import hyperopt

import random as rd                                                              # Random seed generation
import time                                                                      # Time program run time


import matplotlib.pyplot as plt
from matplotlib.patches import Patch                                             # CV visualise
from matplotlib import colors 

from sklearn.metrics import(
    matthews_corrcoef,                                                           # MCC for evaluation
    confusion_matrix,                                                            # Confusion matrix for classification evalutation
    classification_report                                                        # Return the F1, precision, and recall of a prediction
    )

from sklearn.model_selection import(
    GroupKFold                                                                   # K-fold CV with as groups
        )

from sklearn.ensemble import RandomForestClassifier                              # SK learn API for classificastion random forests

from hyperopt import fmin, tpe, hp, Trials, STATUS_OK                                       # Functions for minimising cost functions
from hyperopt.pyll.base import scope
from functools import partial

np.set_printoptions(precision = 3,threshold=np.inf, suppress=True)               # Full array printing

### Split dataset into training and validation sets

In [ ]:
def open_data(file_train, file_test):
    """      
     Returns:    Training_Set     Scaled 80% training set split
                 Testing_Set      Scaled 20% testing set split
            
    Opens the scaled training and testing data
    """
    Training_Set = pd.read_csv(file_train, index_col = 0)
    Testing_Set = pd.read_csv(file_test,index_col = 0)
    
    with open("seed.txt", "r") as f:
        seed = int(f.read().strip())
    
    return Training_Set, Testing_Set, seed

In [ ]:
def learning_data(Training_Set, Testing_Set):
    """      
    Input:      Training_Set     Scaled 80% training set split
                Testing_Set      20% testing set split

    Returns:    TrainData        Training features 
                TrainLabels      Training labels
                TestData         Testing features 
                TestLabels       Testing labels
            
    Separates training and testing data into features and labels
    """
    TrainData     = Training_Set.drop(['AC Code','dataset'], axis =1)  
    TrainLabels   = Training_Set['dataset']
    
    TestData     = Testing_Set.drop(['AC Code','dataset'], axis =1)  
    TestLabels   = Testing_Set['dataset']        
    
    return (TrainData, TrainLabels, TestData, TestLabels)

## Group K-fold CV (outer loop)

In [ ]:
def CV(Training_Set):
    """      
    Input:      Training_Set     80% training set split
            
    Returns:    IT_list         List of training features for each fold
                LT_list         List of training class labels for each fold
                IV_list         List of validation features for each fold
                LV_list         List of validation class labels for each fold

    K-fold CV with protein groups separated between training and validation sets for each fold. Creates 5 folds.
    """
    
    features     = Training_Set.drop(['dataset'], axis =1)         #Features for training
    labels       = Training_Set['dataset']                         #Class labels for training
    groups       = Training_Set['AC Code'].to_list()               #List of proteins for grouping
    
    CV           = GroupKFold(n_splits = 5)                        #Creates 5 splits
    
    IT_list      = []
    LT_list      = []
    IV_list      = []
    LV_list      = []
    
    for train_idx, val_idx in CV.split(features, labels, groups):       #Generates the indices to be used for a training and validation split. Indicies are unique to train/ val sets

        Input_train                        = features.loc[train_idx]    #New dataframe from selected indices
        Classes_train                      = labels.loc[train_idx]
        Input_train.drop(['AC Code'], axis = 1, inplace = True)         #Group identifer not needed for training

                
        Input_val                          = features.loc[val_idx]
        Classes_val                        = labels.loc[val_idx]
        Input_val.drop(['AC Code'], axis   = 1, inplace = True)
        
        Input_train.reset_index(drop = True, inplace = True)            #Reset index of each set for compatability with balancing
        Classes_train.reset_index(drop = True, inplace = True)
        Input_val.reset_index(drop = True, inplace = True)
        Classes_val.reset_index(drop = True, inplace = True)

        IT_list.append(Input_train)       
        LT_list.append(Classes_train)
        IV_list.append(Input_val)
        LV_list.append(Classes_val)
    
    return(IT_list, LT_list, IV_list, LV_list)


## Balancing (inner loop)

In [ ]:
def find_minority_class(classData):
    """ 
    Input:        classData  Array of class labels

    Returns:      minClass   The label for the minority class
                  minSize    The number of items in the minority class
                  maxSize    The number of items in the majority class

    Find information about class size imbalance
    """
    
    Minority_count = 0
    Majority_count = 0
    for datum in classData:
        if datum == 1:
            Majority_count += 1
        elif datum == 0:
            Minority_count += 1

    minClass = 0
    minSize  = Minority_count
    maxSize  = Majority_count
    if Minority_count > Majority_count:
        minClass = 1
        minSize  = Majority_count
        maxSize  = Minority_count

    return minClass, minSize, maxSize

In [ ]:
def balance(inData, classData, minClass, minSize):
    """ 
    Input:        inData          Dataframe of input data
                  classData       Series of classes assigned
                  minorityClass   class label for the minority class
                  minoritySize    size of the minority class

    Returns:      usedLines       array of indexes that are of interest for a balanced dataset

    Perform the actual balancing for a fold between SNPs and PDs
    """
    usedLines = [False] * len(inData)      #Array of false for length of data
    for i in range(len(inData)):
        if classData[i] == minClass:       #Balance directly with dataframe
            usedLines[i] = True            #True lines are SNP
            
    usedCount = 0
    while usedCount < minSize:
        i = rd.randrange(len(inData))
        if usedLines[i] == False:
            usedLines[i] = True
            usedCount += 1                 #Set PD lines "True", until equal to number of SNP lines

    return usedLines

In [ ]:
def balance_data(inData, classData, usedLines):
    """     
    Input:      inData         Datframe of input training data
                classData      Series of classes assigned to training data
                usedLines      Array of line indexes to print

    Returns:    input_balance  Dataframe of balanced training features
                label_balance  Dataframe of balanced training labels
                       
    Create dataframe of the input training data and classes used. Index_list tracks the indicies between usedLines and inData, used to pull the required5 lines.
    """
    input_balance = []
    label_balance = []
    index_list = []
    
    for i in range(len(usedLines)):
        if usedLines[i] == True:
            index_list.append(i)
             
    input_balance = inData.iloc[index_list].reset_index(inplace = False, drop = True)
    label_balance = classData.iloc[index_list].reset_index(inplace = False, drop = True) 
    
    return input_balance, label_balance

### Balance for n folds

In [ ]:
def Balance_ratio(maxSize, minSize): 
    """ 
    Input:      maxSize     The number of items in the majority class
                minSize     The number of items in the minority class

    Returns:    BF          Number of balancing folds

    Calculate the number of balancing folds needed using ratio of majority to minority class size. Double to ensure sufficient
    majority class instances are sampled, then + 1 to make odd to allow weighted vote.
    """
    Divide = maxSize/minSize
    BF = ((2 * round(Divide)) + 1)    # ratio to nearest odd integer
    return BF

In [ ]:
def Balance_Folds(BF, inData, classData, minClass, minSize):
    """ 
    Input:      BF                Number of balancing folds
                inData            Datframe of input training data
                classData         Series of classes assigned to training data
                minClass          The label for the minority class
                minSize           The number of items in the minority class
                                  
    Returns:    Input_folds       List of balanced training feature folds
                Output_folds      List of balanced training label folds

    Runs balance_data() for the number of balancing folds. Return lists of balanced folds features and labels
    where each item is the output of balance_data()
    """
    Input_folds  = []
    Output_folds = []

    for i in range(BF):
        usedLines                    = balance(inData, classData, minClass, minSize)
        input_balance, label_balance = balance_data(inData, classData, usedLines)
        
        Input_folds.append(input_balance)
        Output_folds.append(label_balance)
            
    return Input_folds, Output_folds

### RFC hyperparameter tuning

In [ ]:
def hyperopt_space():
    """ 
    Returns:   Space         Parameter space for hyperparameter searching

    Define paramater space for Hyperopt to search throug
   """  
    space = {
        'n_estimators': scope.int(hp.quniform('n_estimators', 100, 1000, 100)),
        'max_depth': scope.int(hp.quniform('max_depth', 3, 9, 1)),
        'eta': hp.uniform('eta', 0.45, 0.75),
        }
     
    return space


In [ ]:
def objective(params, Input_folds, Output_folds): 
    """ 
    Input:      params            Search paramaters parsed in fmin function()                     
                Input_folds       List of balanced training feature folds
                Output_folds      List of balanced training label folds
                
    Returns:    loss, status      The MCC from evaluating hyperparameters during search

    Define the model that Hyperopt will optimise hyperparameters for
    """     
    max_depth = params['max_depth']
    eta = params['eta']
    num_boost_round = num_boost_round['num_boost_round']
    
    settings = {
        'booster': 'gbtree',
        'objective': 'binary:logistic',
        'disable_default_eval_metric': 1,
        'verbosity': 1,
        'max_depth': max_depth,
        'eta': eta,
            } 
                    
    model = RFC.fit(settings)
    #Generates and fits a GBC for each training balanced fold
    
    pred = model.predict(Output_folds)
    MCC = matthews_corrcoef(Output_folds.get_label(), pred.round())
                                                                         
    return {'loss': -MCC, 'status': STATUS_OK}


### Train RFC on the trainings folds

In [ ]:
def BF_fitting(BF, Input_folds, Output_folds): 
    """ 
    Input:      BF                Number of balancing folds                      
                Input_folds       List of balanced training feature folds
                Output_folds      List of balanced training label folds

    Returns:    BF_RFC            List of RFCs trained on each balancing fold

    Create RFC model that returns probability predictions for each fold, using output of Balance_Folds() as training data
    """    
    BF_RFC = []
    for i in range(BF):
        BF_RFC.append(RandomForestClassifier(verbose = 0)) #Generates a RFC for each fold's training data
        BF_RFC[i].fit(Input_folds[i], Output_folds[i])     #Fits the RFC to each folds' training data
        
    return BF_RFC

#### Validate each RFC on validation set, for each fold

In [ ]:
def BF_validate(BF_RFC, ValData):
    """ 
    Input:      BF_RFC          List of RFCs trained on balancing folds
                ValData         Unseen validation features from CV fold
                
    Returns:    Prob_matrix     List of arrays. Each item is 2D matrix where the 1st dimension is each subset in balancing fold, 
                                2nd dimension is predicted probability
    
    Test the trained RFCs on the test set, then for every instance, outputs the predicted probability for each class
    """
    
    Prob_matrix = []
    
    for i in range(len(BF_RFC)):
        Prob = BF_RFC[i].predict_proba(ValData.values) #Predicts the probability of an instance belonging to major or minor class
        Prob_matrix.append(Prob)   
        
    return Prob_matrix

### Weighted voting

In [ ]:
def Weighted_Vote(Prob_matrix):
    """ 
    Input:      Prob_matrix     List of arrays. 2D matrix where the 1st dimension is each subset in balancing fold, 
                                2nd dimension is predicted probability

    Returns:    Final_vote      Weighted vote classification

    Calculate the final predictions with weighted vote using confidence scores. 
    Sc = (S0 -T)/(1-T) if S0> T
    Sc = (T-S0)/T if S0 < T
    """
    Sc_SNP = []
    Sc_PD = []
    
    for i in range(len(Prob_matrix)):
        Sc_SNP.append(Prob_matrix[i][:,0])
        Sc_PD.append(Prob_matrix[i][:,1])
    
    # T = 0.5                                     #Lower threshold gives more sensitivity to PDs over SNPs
    # Sc_SNP = []
    # Sc_PD = []

    # for fold in range(len(Prob_matrix)):        #Calculates SNP Sc all instances in each fold

    #     Sc_SNP_fold = []                        #List of the Sc for each fold
    #     for value in range(len(Prob_matrix[fold][:,0])):
    #         S0 = Prob_matrix[fold][:,0][value]  #Each SNP's confidence in prob matrix fold
    #         if S0 < T:
    #             Sc = (T - S0)/T
    #         elif S0 >= T:
    #             Sc = (S0 - T)/(1 - T)        
    #         Sc_SNP_fold.append(Sc)              #List of Sc for each fold
    #     Sc_SNP.append(Sc_SNP_fold)              #List of folds with Sc

    # for fold in range(len(Prob_matrix)):        #Calculates PD Sc all instances in each fold
    #     Sc_PD_fold = []
    #     for value in range(len(Prob_matrix[fold][:,1])):
    #         S0 = Prob_matrix[fold][:,1][value]  #Each PD's confidence in prob matrix fold
    #         if S0 < T:
    #             Sc = (T - S0)/T
    #         elif S0 >= T:
    #             Sc = (S0 - T)/(1 - T)        
    #         Sc_PD_fold.append(Sc)
    #     Sc_PD.append(Sc_PD_fold)
        
    columnSNP = np.stack(Sc_SNP)                #Covert list of lists to array, shape (5,~539)
    columnPD  = np.stack(Sc_PD)

    Sum_SNP   = np.sum(columnSNP, axis = 0)     #Sum of all SNP confidence scores. 1D Array
    Sum_PD    = np.sum(columnPD, axis = 0)      #Sum of all PD confidence scores. 1D Array
    
    
    Vote_arr  = [] 

    for i in range(len(Sum_PD)):
        if Sum_PD[i] >= Sum_SNP[i]:
            Vote_arr.append([1])                #Append PD classifications to list
        elif Sum_SNP[i] > Sum_PD[i]:
            Vote_arr.append([0])                #Append SNP classifications to list

    Final_vote = np.stack(Vote_arr)             #Converts list of arrays to a 2D array
    Final_vote = Final_vote.ravel()             #Flattens 2D array to 1D array

    return(Final_vote, Sum_PD, Sum_SNP)         #Returns the final confidence scores


In [ ]:
def evalutation(Vallabel, Final_vote):
    """ 
    Input:      Vallabel           Unseen validation class labels from CV fold
                Final_vote         Weighted vote classification

    Evaluate each fold with confusion matrix and MCC
    """
    Output_pred = Final_vote
    print(f"-----------------------------------------------------\n              ***Fold {folds + 1} Evaluation***\n")
    print(f"Confusion Matrix:\n {confusion_matrix(Vallabel, Output_pred)}")
    print(f"{classification_report(Vallabel, Output_pred)}\nMCC                {matthews_corrcoef(Vallabel, Output_pred)}\n")

### Main Program

In [ ]:
file = "AC_dataset.csv"

# file_train                = input("Enter file for training: ")
# file_test                 = input("Enter file for testing: ")
file_train                  = "STraining_Set.csv"
file_test                   = "STesting_Set.csv"
Training_Set, Testing_Set, seed = open_data(file_train, file_test)                                #Create training and testing sets
rd.seed(seed)
IT_list, LT_list, IV_list, LV_list = CV(Training_Set, seed)                                       #Cross-validate training set

for folds in range(len(IT_list)):                                                       
    classData                   = LT_list[folds]                                            #Training labels
    inData                      = IT_list[folds]                                            #Training features
    ValData                     = IV_list[folds]                                            #Validation features
    Vallabel                    = LV_list[folds]                                            #Validation labels

    minClass, minSize, maxSize  = find_minority_class(classData)                            #Determines imbalance
    BF                          = Balance_ratio(maxSize, minSize)                           #Determins number of balancing folds needed
    Input_folds, Output_folds   = Balance_Folds(BF, inData, classData, minClass, minSize)   # balance() and balance_data() functions are called under this
    
    # space                   = hyperopt_space()
    # trials                  = Trials()
    # fmin_objective          = partial(objective,
    #                                     d_train_list    = d_train_list[randrange(BF)],
    #                                     d_val           = d_val,
    #                                     MCC_eval_metric = MCC_eval_metric)
    # best                    = fmin(fn = fmin_objective,
    #                                 space             = space,
    #                                 algo              = tpe.suggest,
    #                                 max_evals         = 120,
    #                                 trials            = trials,
    #                                 )
    
    # final_param.append(trials.argmin)
    
    BF_RFC                      = BF_fitting(BF, Input_folds, Output_folds)
    Prob_matrix                 = BF_validate(BF_RFC, ValData)

    Final_vote, Sum_PD, Sum_SNP = Weighted_Vote(Prob_matrix)

    evalutation(Vallabel, Final_vote)